In [1]:
# saves you having to use print as all exposed variables are printed in the cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# suppress warning message
# import warnings
# warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import torch
import pyarrow as pa
import pyarrow.compute as pc

---------

In [4]:
from nn_rag import Knowledge
from nn_rag.components.commons import Commons

/opt/anaconda3/envs/neural/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-07-13 15:17:34.324083: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# create instance of capability
kn = Knowledge.from_memory()

In [10]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("all-mpnet-base-v2")

sentences = [
    "The weather is so nice!",
    "It's so sunny outside.",
    "He's driving to the movie theater.",
    "She's going to the cinema.",
]
embeddings = model.encode(sentences, normalize_embeddings=True)
model.similarity(embeddings, embeddings)
result = util.pairwise_cos_sim(embeddings[::2], embeddings[1::2])

result

/opt/anaconda3/envs/neural/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tensor([[1.0000, 0.7235, 0.0290, 0.1309],
        [0.7235, 1.0000, 0.0613, 0.1129],
        [0.0290, 0.0613, 1.0000, 0.5027],
        [0.1309, 0.1129, 0.5027, 1.0000]])

tensor([0.7235, 0.5027])